In [ ]:
kf = KFold(n_splits=5, shuffle=True)
df_res = pd.DataFrame(dtype = 'object')

dict_param_grid = {'KNN': {'n_neighbors': [1,2,3,5,7,9,11]}}

In [ ]:
# extracting parameters
reg = KNeighborsRegressor()
name_reg = 'KNN'
param_grid = {str(reg)[:-2].lower()+'__'+k : v \
              for k,v in dict_param_grid[name_reg].items()}

In [ ]:
# researching best hyperparameters and fitting on training set
pipe = make_pipeline(column_trans, reg)
gscv = GridSearchCV(pipe,
                    param_grid=param_grid,
                    cv=kf, verbose=1)
gscv.fit(X_tr,y1_tr)

# best hyperparams
df_res = pd.DataFrame(dtype = 'object')
df_res.loc['name_params', name_reg] = tuple([k for k,v in dict_param_grid[name_reg].items()])
df_res.loc['best_params', name_reg] = tuple([gscv.best_params_[p] for p in param_grid])

# score of the model with best params on testing set
y_pr = gscv.predict(X_te)
res = scores_reg(name_reg, X_te, y1_te, y_pr).astype('object')
df_res = df_res.append(res.to_frame())

# mean cv score of the model with best params on testing set
res = cv_scores_reg(name_reg, gscv.best_estimator_, X_te, y1_te, cv=6).astype('object')
df_res = df_res.append(res.to_frame())
df_res

In [ ]:
# plotting optimisation curve of hyperparameters


list_n_params = [n for n in gscv_res.keys()if 'param_' in n]
list_params = [gscv.cv_results_[p].tolist() for p in list_n_params]
for n_p, li_p in zip(list_n_params, list_params):
    plt.errorbar(li_p,
            gscv.cv_results_['mean_test_score'],
            yerr=gscv.cv_results_['std_test_score'])
    plt.gcf().suptitle(n_p)
# plt.gca().set_xscale('log')
# plt.gca().set(xlim=(0,10000))

In [ ]:
reg = LinearRegression()
name_reg = 'LinReg(allcols)'

pipe = make_pipeline(column_trans, reg)
pipe.fit(X_tr,y1_tr)

y_pr = pipe.predict(X_te)

res = scores_reg(name_reg, X_te, y1_te, y_pr).astype('object')
df_lr = df_lr.append(res.to_frame())
res = cv_scores_reg(name_reg, pipe,
                    X_te, y1_te, cv=6).astype('object')
df_lr = df_lr.append(res.to_frame())

df_lr

In [ ]:
# plotting optimisation curve of hyperparameters
param_grid = dict_param_grid[name_reg]

gscv_res = knn_gscv.cv_results_
list_n_params = [n for n in gscv_res.keys()if 'param_' in n]
list_params = [gscv_res[p].tolist() for p in list_n_params]
for n_p, li_p in zip(list_n_params, list_params):
    plt.errorbar(li_p,
            gscv_res['mean_test_score'],
            yerr=gscv_res['std_test_score'])
    plt.gcf().suptitle(n_p)
# plt.gca().set_xscale('log')
# plt.gca().set(xlim=(0,10000))

In [ ]:
reg = ElasticNet()
name_reg = 'my_enet'
dict_param_grid = {'my_enet': {'alpha': [1,15],
                               'l1_ratio': [0.3,0.5]}}
kf_gscv = KFold(n_splits=2, shuffle=True)

# version unique
enet_gscv, new_df_res = model_optimizer(name_reg=name_reg, data_preproc=column_trans,
                                 reg=reg, param_grid=dict_param_grid[name_reg],
                                 Xtr=X_tr, ytr=y1_tr, Xte=X_te, yte=y1_te,
                                 cv_gs=kf_gscv, cv_test=6)
df_res = pd.concat([df_res, new_df_res], axis=1)

display(df_res)

plot_2D_hyperparam_opt(enet_gscv)

In [ ]:
reg = LinearRegression()
name_reg = 'LinReg(allcols)'

pipe = make_pipeline(column_trans, reg)
pipe.fit(X_tr,y1_tr)

y_pr = pipe.predict(X_te)

ser = scores_reg(name_reg, X_te_sel, y1_te, y_pr).astype('object')
ser = ser.append(cv_scores_reg(name_reg, pipe, X_te_sel, y1_te, cv=6).astype('object'))

df_lr = df_lr.append(ser.to_frame())

df_lr

In [ ]:
# name_reg = name_reg
# pipe= pipe
# Xte = X_te
# yte = y1_te
# df_res=df_lr
# cv=5

# if df_res is None:
#     df_res = pd.DataFrame(dtype = 'object')
# df_res_mod = pd.DataFrame(dtype = 'object')
# ypr = pipe.predict(Xte)
# ser = scores_reg(name_reg, Xte, yte, ypr).astype('object')
# ser = ser.append(cv_scores_reg(name_reg, pipe, Xte, yte, cv=cv).astype('object'))
# df_res_mod = pd.concat([df_res,ser.to_frame()],1)
# df_res_mod

In [ ]:
def plot_append_curve(x, y, c, fig=None):
    %matplotlib
    if fig is None:
        fig = plt.figure()
    ax = fig.axes[0]
    ax.plot(x, y, c=c)
    %matplotlib inline
    return fig

fig = plot_append_curve([100,11,90,13], [100,90,70,200], c='grey', fig=fig);
fig

In [ ]:
# OLD VERSION
class CustEncoder(BaseEstimator) :

    def __init__(self, l_card_cols, h_card_cols, bool_cols,
                 strat_low_card='ord', strat_high_card='ord'):
        self.l_card_cols = l_card_cols
        self.h_card_cols = h_card_cols
        self.bool_cols = bool_cols
        self.strat_low_card = strat_low_card
        self.strat_high_card = strat_high_card

    def fit(self, X, y=None):
        # Dictionary to translate strategies
        d_enc = {'ohe': ce.OneHotEncoder(),
                 'hash': ce.HashingEncoder(),
                 'ord': ce.OrdinalEncoder(),
                 'loo': ce.LeaveOneOutEncoder(),
                 'bin': ce.BinaryEncoder()}

        # Creates a columns transformer with chosen strategies
        self.column_trans = ColumnTransformer([("low_card", d_enc[self.strat_low_card], l_card_cols),
                                       ("bool", ce.OrdinalEncoder(), bool_cols),
                                       ("high_card", d_enc[self.strat_high_card], h_card_cols)],
                                       remainder=StandardScaler())
        return self.column_trans.fit(X, y)
  
    def transform(self, X, y=None):
        return  self.column_trans.transform(X)

In [ ]:
results = knn_gscv3.cv_results_
scoring = ['neg_root_mean_squared_error', 'r2']
param = 'param_KNN3__n_neighbors'


plt.figure(figsize=(8, 5))
plt.title("GridSearchCV evaluating using multiple scorers simultaneously",
          fontsize=16)
plt.xlabel("n_neighbors")
plt.ylabel("Score")
ax = plt.gca()

# Get the regular numpy array from the MaskedArray
X_axis = np.array(results[param].data, dtype=float)

for scorer, color in zip(sorted(scoring), ['g', 'k']):
    for sample, style in (('train', '--'), ('test', '-')):
        sample_score_mean = results['mean_%s_%s' % (sample, scorer)]
        sample_score_std = results['std_%s_%s' % (sample, scorer)]
        ax.fill_between(X_axis, sample_score_mean - sample_score_std,
                        sample_score_mean + sample_score_std,
                        alpha=0.1 if sample == 'test' else 0, color=color)
        ax.plot(X_axis, sample_score_mean, style, color=color,
                alpha=1 if sample == 'test' else 0.7,
                label="%s (%s)" % (scorer, sample))

    best_index = np.nonzero(results['rank_test_%s' % scorer] == 1)[0][0]
    best_score = results['mean_test_%s' % scorer][best_index]

    # Plot a dotted vertical line at the best score for that scorer marked by x
    ax.plot([X_axis[best_index], ] * 2, [0, best_score],
            linestyle='-.', color=color, marker='x', markeredgewidth=3, ms=8)

    # Annotate the best score for that scorer
    ax.annotate("%0.2f" % best_score, 
                (X_axis[best_index]+0.5, best_score + 0.005))

plt.legend(loc="best")
plt.grid(False)
plt.show()

In [ ]:
class CustTransformer(BaseEstimator) :

    def __init__(self, thresh_card=12,
                 strat_binary = 'ord', strat_low_card ='ohe',
                 strat_high_card ='hash', strat_quant = 'stand'):
        self.thresh_card = thresh_card
        self.strat_binary = strat_binary
        self.strat_low_card = strat_low_card
        self.strat_high_card = strat_high_card
        self.strat_quant = strat_quant

    def d_type_col(self, X):
        bin_cols = (X.nunique()[X.nunique()==2].index)
        X_C_cols = X.select_dtypes(include=['object', 'category'])
        C_l_card_cols = X_C_cols.nunique()[X_C_cols.nunique()<self.thresh_card].index
        C_h_card_cols = X_C_cols.nunique()[X_C_cols.nunique()>=self.thresh_card].index
        Q_cols = [c for c in X.select_dtypes(include=[np.number]).columns\
                                                        if c not in bin_cols]
        d_t = {'binary': bin_cols,
               'low_card': C_l_card_cols,
               'high_card': C_h_card_cols,
               'remaind': Q_cols}
        return d_t

    def fit(self, X, y=None):
        # Dictionary to translate strategies
        d_enc = {'ohe': ce.OneHotEncoder(),
                 'hash': ce.HashingEncoder(),
                 'ord': ce.OrdinalEncoder(),
                 'loo': ce.LeaveOneOutEncoder(),
                 'bin': ce.BinaryEncoder(),
                 'stand': StandardScaler(),
                 'minmax': MinMaxScaler(),
                 'maxabs': MaxAbsScaler(),
                 'robust': RobustScaler(quantile_range=(25, 75)),
                 'norm': Normalizer(),
                 'quant_uni': QuantileTransformer(output_distribution='uniform'),
                 'quant_norm': QuantileTransformer(output_distribution='normal'),
                 'pow': PowerTransformer(method='yeo-johnson'), # 'boxcox'
                 }
        # Creates a columns transformer with chosen strategies
        self.column_trans = \
                ColumnTransformer([("binary", d_enc[self.strat_binary],
                                    self.d_type_col(X)['binary']),
                                   ("low_card", d_enc[self.strat_low_card],
                                    self.d_type_col(X)['low_card']),
                                   ("high_card", d_enc[self.strat_high_card],
                                    self.d_type_col(X)['high_card']),
                                   ("remaind", d_enc[self.strat_quant],
                                    self.d_type_col(X)['remaind'])])
        return self.column_trans.fit(X, y)
  
    def transform(self, X, y=None):
        return  self.column_trans.transform(X)
''' create a pipeline with datapreprocessing column transformer and regressor
    then searches for best hyperparameters with gscv
    then stores best parameters
    then computes the scores of the model on testing set
    then computes the cv scores of the model on testing set'''

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import RandomizedSearchCV

def model_optimizer(data_preproc, name_reg, reg, param_grid,
                    Xtr, ytr, Xte, yte,
                    cv_search=5, groups=None, cv_test=6,
                    gs_score='neg_root_mean_squared_error',
                    search_strat='grid', n_iter=10):

    pipe = Pipeline([('preproc', data_preproc),
                    (name_reg, reg)])
    
    # researching best hyperparameters and fitting on training set
    if search_strat=='grid':
        scv = GridSearchCV(pipe, param_grid = param_grid,
                           cv=cv_search, scoring=gs_score, verbose=1)
        print("grid")
    elif search_strat=='rand':
        scv = RandomizedSearchCV(pipe, param_distributions = param_grid,
                            cv=cv_search, n_iter= n_iter,
                            scoring=gs_score, verbose=1)
        print("randomized")
    else:
        print("ERROR: This strategy of hyperparameter tuning does not exist.")
    scv.fit(Xtr, ytr, groups=groups) # to stratify the folds using a GroupFolds

    # best hyperparams
    df_res = pd.DataFrame(dtype = 'object')
    df_res.at['name_params', name_reg] =\
                    str(list(param_grid.keys()))
    df_res.at['best_params', name_reg] =\
                    str([scv.best_params_[p] for p in param_grid])

    # score of the model with best params on testing set
    ypr = scv.predict(Xte)
    res = scores_reg(name_reg, Xte, yte, ypr,
    	             exclude=['Adj_R2']).astype('object')
    df_res = df_res.append(res.to_frame())

    # mean cv score of the model with best params on testing set
    res = cv_scores_reg(name_reg, scv.best_estimator_,
                        Xte, yte, cv=cv_test).astype('object')
    df_res = df_res.append(res.to_frame())

    return scv, df_res

In [ ]:
small_df = df[['Outlier', 'Neighborhood', 'CertifiedPreviousYear',
               'NumberofFloors','ExtsurfVolRatio']]
small_df.head(2)

In [ ]:
cust_trans = ColumnTransformer([
                                ("bin",  ce.BinaryEncoder(), ['CertifiedPreviousYear']),
                                ("ohe", ce.BinaryEncoder(), ['Outlier']),
                                ("hash", ce.BinaryEncoder() , ['Neighborhood']),
                                # ("num", StandardScaler(), ['NumberofFloors','ExtsurfVolRatio']),
                                ],
                               remainder = 'passthrough')
df_enc = cust_trans.fit_transform(small_df)
cust_trans.get_feature_names()

In [ ]:
 small_df.shape, df_enc.shape

In [ ]:
def d_type_col(X, thresh_card=6):
    bin_cols = (X.nunique()[X.nunique()<=2].index)
    X_C_cols = X.select_dtypes(include=['object', 'category'])
    C_l_card_cols = X_C_cols.nunique()[X_C_cols.nunique().between(3, thresh_card)].index
    C_h_card_cols = X_C_cols.nunique()[X_C_cols.nunique()>thresh_card].index
    Q_cols = [c for c in X.select_dtypes(include=[np.number]).columns\
                                                    if c not in bin_cols]
    d_t = {'binary': bin_cols,
            'low_card': C_l_card_cols,
            'high_card': C_h_card_cols,
            'numeric': Q_cols}
    return d_t

In [ ]:
column_trans =  DataFrameMapper([
                                 (d_type_col(data)['binary'], ce.OrdinalEncoder()),
                                 (d_type_col(data)['low_card'], ce.OneHotEncoder()),
                                 (d_type_col(data)['high_card'], ce.OrdinalEncoder()),
                                 (d_type_col(data)['numeric'], StandardScaler())
                                 ],
                                df_out=False)

In [ ]:
data = pd.DataFrame({'pet':      ['cat', 'dog', 'dog', 'fish', 'cat', 'dog', 'cat', 'fish'],
                     'children': [4., 6, 3, 3, 2, 3, 5, 4],
                     'index': ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'],
                     'choice': ['0', '1', '1', '1', '0', '0', '1', '0'],
                     'salary':   [90., 24, 44, 27, 32, 59, 36, 27]})
data, d_type_col(data)

In [ ]:
X = data

ct_cat =  ColumnTransformer([
                                 ('binary', ce.OrdinalEncoder(), d_type_col(X)['binary']),
                                 ('low_card', ce.OneHotEncoder(), d_type_col(X)['low_card']),
                                 ('high_card', ce.HashingEncoder(), d_type_col(X)['high_card']),
                                #  ('numeric', StandardScaler(), d_type_col(X)['numeric'])
                                 ], remainder='passthrough')

num_cols = d_type_col(X)['numeric']
num_trans = Pipeline([("numeric", StandardScaler())])

cat_cols = d_type_col(X)['binary'].union(d_type_col(X)['low_card']).union(d_type_col(X)['high_card'])
cat_trans = Pipeline([("categ", ct_cat)])

column_trans =  ColumnTransformer([
                                   ('cat', cat_trans, cat_cols),
                                   ('num', num_trans, num_cols),
                                   ], remainder='passthrough')
                                                
column_trans.fit(data)
cat_trans.fit(data)
ct_cat.get_feature_names()

In [ ]:
from sklearn.base import BaseEstimator
from sklearn.compose import ColumnTransformer
import category_encoders as ce
from sklearn.preprocessing import *

class CustTransformer(BaseEstimator) :

    def __init__(self, thresh_card=12,
                 strat_binary = 'ord', strat_low_card ='ohe',
                 strat_high_card ='hash', strat_quant = 'stand'):
        self.thresh_card = thresh_card
        self.strat_binary = strat_binary
        self.strat_low_card = strat_low_card
        self.strat_high_card = strat_high_card
        self.strat_quant = strat_quant

    def d_type_col(self, X):
        bin_cols = (X.nunique()[X.nunique()==2].index)
        X_C_cols = X.select_dtypes(include=['object', 'category'])
        C_l_card_cols = X_C_cols.nunique()[X_C_cols.nunique()<self.thresh_card].index
        C_h_card_cols = X_C_cols.nunique()[X_C_cols.nunique()>=self.thresh_card].index
        Q_cols = [c for c in X.select_dtypes(include=[np.number]).columns\
                                                        if c not in bin_cols]
        d_t = {'binary': bin_cols,
               'low_card': C_l_card_cols,
               'high_card': C_h_card_cols,
               'remaind': Q_cols}
        return d_t

    def fit(self, X, y=None):
        # Dictionary to translate strategies
        d_enc = {'ohe': ce.OneHotEncoder(),
                 'hash': ce.HashingEncoder(),
                 'ord': ce.OrdinalEncoder(),
                 'loo': ce.LeaveOneOutEncoder(),
                 'bin': ce.BinaryEncoder(),
                 'stand': StandardScaler(),
                 'minmax': MinMaxScaler(),
                 'maxabs': MaxAbsScaler(),
                 'robust': RobustScaler(quantile_range=(25, 75)),
                 'norm': Normalizer(),
                 'quant_uni': QuantileTransformer(output_distribution='uniform'),
                 'quant_norm': QuantileTransformer(output_distribution='normal'),
                 'pow': PowerTransformer(method='yeo-johnson'), # 'boxcox'
                 }
        # Creates a columns transformer with chosen strategies
        self.column_trans = \
                ColumnTransformer([("binary", d_enc[self.strat_binary],
                                    self.d_type_col(X)['binary']),
                                   ("low_card", d_enc[self.strat_low_card],
                                    self.d_type_col(X)['low_card']),
                                   ("high_card", d_enc[self.strat_high_card],
                                    self.d_type_col(X)['high_card']),
                                   ("remaind", d_enc[self.strat_quant],
                                    self.d_type_col(X)['remaind'])])
        return self.column_trans.fit(X, y)
  
    def transform(self, X, y=None):
        return  self.column_trans.transform(X)

In [ ]:
df_outliers = df[df['Outlier']!='not']
df_outliers_h = df[df['Outlier']=='high']
df_outliers_l = df[df['Outlier']=='low']
sns.scatterplot(data = df, x='PropertyGFATotal', y= 'SiteEnergyUseWN(kBtu)', color='k', size=1, edgecolor=None)
sns.scatterplot(data = df_outliers, x='PropertyGFATotal', y= 'SiteEnergyUseWN(kBtu)',
                hue='Outlier', size=1, edgecolor=None)
plt.gca().legend(loc=2)

In [ ]:
def plot_perm_importance(model, X, y, scoring='r2'):

    results = permutation_importance(model, X, y, scoring=scoring) 
    df_ = pd.DataFrame(results.importances_mean,
                       index = X.columns, columns=[name_reg])

    fig, ax = plt.subplots()
    df_[name_reg].sort_values(ascending=False).plot.bar(color='grey')
    fig.set_size_inches(12,3)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha="right" )
    return df_

In [ ]:
x

In [ ]:
class SampleAndFeatureFilter(BaseEstimator, TransformerMixin):
    def __init__(self, perc = None):
        self.perc = perc

    def fit(self, X, y=None):
        self.X = X
        sum_per_feature = X.sum(0)
        sum_per_sample = X.sum(1)
        self.featurefilter = sum_per_feature >= np.percentile(sum_per_feature, self.perc)
        self.samplefilter = sum_per_sample >= np.percentile(sum_per_sample, self.perc)
        return self

    def transform(self, X, y=None, copy=None):
        idx = id(X)
        X=X[:,self.featurefilter]
        if idx == id(self.X):
            X = X[self.samplefilter, :]
            if y is not None:
                y = y[self.samplefilter]
            return X, y
        return X

    def fit_transform(self, X, y=None, **fit_params):
        if y is None:
            return self.fit(X, **fit_params).transform(X)
        else:
            return self.fit(X, y, **fit_params).transform(X,y)

class Pipeline_Xy(Pipeline):
    def _pre_transform(self, X, y=None, **fit_params):
        fit_params_steps = dict((step, {}) for step, _ in self.steps)
        for pname, pval in items(fit_params):
            step, param = pname.split('__', 1)
            fit_params_steps[step][param] = pval
        Xt = X
        yt = y
        for name, transform in self.steps[:-1]:
            if hasattr(transform, "fit_transform"):
                res = transform.fit_transform(Xt, yt, **fit_params_steps[name])
                if len(res) == 2:
                    Xt, yt = res
                else:
                    Xt = res
            else:
                Xt = transform.fit(Xt, y, **fit_params_steps[name]) \
                              .transform(Xt)
        return Xt, yt, fit_params_steps[self.steps[-1][0]]

    def fit(self, X, y=None, **fit_params):
        Xt, yt, fit_params = self._pre_transform(X, y, **fit_params)
        self.steps[-1][-1].fit(Xt, yt, **fit_params)
        return self

    def fit_transform(self, X, y=None, **fit_params):
        Xt, yt, fit_params = self._pre_transform(X, y, **fit_params)
        if hasattr(self.steps[-1][-1], 'fit_transform'):
            return self.steps[-1][-1].fit_transform(Xt, yt, **fit_params)
        else:
            return self.steps[-1][-1].fit(Xt, yt, **fit_params).transform(Xt)


In [ ]:
### COEFF IMPORTANCE OF THE FEATURES

# Getting the names of the transformed columns
step_ct = dict_models[name_reg].best_estimator_.named_steps['preproc'].named_steps['cust_trans']
col_names = step_ct.get_feature_names() #dict_scv_params['X']
# Getting the list of the coefficients
col_coefs = dict_models[name_reg].best_estimator_.named_steps[name_reg].coef_

ser = pd.Series(col_coefs, index = col_names)

fig, ax = plt.subplots()
ser.sort_values(ascending=False).plot.bar(color='red');
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha="right" )
fig.set_size_inches(15,3)
plt.show()